<a href="https://colab.research.google.com/github/reformship/kaggle/blob/master/attrition_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

train_file = 'train.csv';
test_file = 'test.csv';
train_employee_data = pd.read_csv(train_file, index_col='user_id');
test_employee_data = pd.read_csv(test_file, index_col='user_id');
print(train_employee_data.head());

from sklearn.model_selection import train_test_split
y = train_employee_data['Attrition'];
X = train_employee_data.drop(['Attrition'], axis=1);

#training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0);

#numerical columns and categorical columns
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']];
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object'];

#count the unique items in each categorical columns
print(train_employee_data[categorical_cols].nunique());

from sklearn.preprocessing import OneHotEncoder
#Use one-hot encoder to encode the categorical columns
OH_encoder = OneHotEncoder(sparse=False);
OH_train_cols = pd.DataFrame(OH_encoder.fit_transform(X_train[categorical_cols]));
OH_valid_cols = pd.DataFrame(OH_encoder.transform(X_val[categorical_cols]));
OH_test_cols = pd.DataFrame(OH_encoder.transform(test_employee_data[categorical_cols]));

OH_train_cols.index = X_train.index;
OH_valid_cols.index = X_val.index;
OH_test_cols.index = test_employee_data.index;

numerical_train_cols = X_train.drop(categorical_cols, axis=1);
numerical_valid_cols = X_val.drop(categorical_cols, axis=1);
numerical_test_cols = test_employee_data.drop(categorical_cols, axis=1);

OH_train = pd.concat([OH_train_cols, numerical_train_cols], axis=1);
OH_valid = pd.concat([OH_valid_cols, numerical_valid_cols], axis=1);
OH_test = pd.concat([OH_test_cols, numerical_test_cols], axis=1);

         Age Attrition  ... YearsSinceLastPromotion  YearsWithCurrManager
user_id                 ...                                              
1374      58        No  ...                       0                     0
1092      45        No  ...                       0                     3
768       40        No  ...                       7                     5
569       36        No  ...                       0                     9
911       25       Yes  ...                       1                     0

[5 rows x 35 columns]
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64


In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder();
y_train = label_encoder.fit_transform(y_train);
print(y_val);
y_val = label_encoder.transform(y_val);
print(y_val);

from sklearn.linear_model import LogisticRegression
#Use LR to fit the data
lr = LogisticRegression(C=1e5);
lr.fit(OH_train, y_train);
my_valid_predict = lr.predict(OH_valid);

#error
false_number = ((my_valid_predict - y_val) ** 2).sum();
total_number = len(y_val);
print((total_number - false_number) / total_number);

import numpy as np
test_preds = lr.predict_proba(OH_test)[1];
print(test_preds);
print(test_employee_data.index);
output = pd.DataFrame({'user_id': test_employee_data.index, 'Attrition': test_preds});
output.to_csv('submission.csv', index=False);
print(output)

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder();
y_train = label_encoder.fit_transform(y_train);
print(y_val);
y_val = label_encoder.transform(y_val);
print(y_val);

from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
#Use LR to fit the data
lr = RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.3, min_samples_leaf=14, min_samples_split=5, n_estimators=100);
lr.fit(OH_train, y_train);
my_valid_predict = lr.predict(OH_valid);

#error
false_number = ((my_valid_predict - y_val) ** 2).sum();
total_number = len(y_val);
print((total_number - false_number) / total_number);

import numpy as np
test_preds = lr.predict_proba(OH_test)[:,1];
print(test_preds);
print(test_employee_data.index);
output = pd.DataFrame({'user_id': test_employee_data.index, 'Attrition': test_preds});
output.to_csv('submission4.csv', index=False);
print(output)

[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8686440677966102
[0.10466009 0.0744741

In [0]:
pip install tpot

In [13]:

import pandas as pd
train_file = 'train.csv';
test_file = 'test.csv';
train_employee_data = pd.read_csv(train_file, index_col='user_id');
test_employee_data = pd.read_csv(test_file, index_col='user_id');
print(train_employee_data.head());

from sklearn.model_selection import train_test_split
y = train_employee_data['Attrition'];
X = train_employee_data.drop(['Attrition'], axis=1);

#training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0);

#numerical columns and categorical columns
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']];
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object'];

#count the unique items in each categorical columns
print(train_employee_data[categorical_cols].nunique());

from sklearn.preprocessing import OneHotEncoder
#Use one-hot encoder to encode the categorical columns
OH_encoder = OneHotEncoder(sparse=False);
OH_train_cols = pd.DataFrame(OH_encoder.fit_transform(X_train[categorical_cols]));
OH_valid_cols = pd.DataFrame(OH_encoder.transform(X_val[categorical_cols]));
OH_test_cols = pd.DataFrame(OH_encoder.transform(test_employee_data[categorical_cols]));

OH_train_cols.index = X_train.index;
OH_valid_cols.index = X_val.index;
OH_test_cols.index = test_employee_data.index;

numerical_train_cols = X_train.drop(categorical_cols, axis=1);
numerical_valid_cols = X_val.drop(categorical_cols, axis=1);
numerical_test_cols = test_employee_data.drop(categorical_cols, axis=1);

OH_train = pd.concat([OH_train_cols, numerical_train_cols], axis=1);
OH_valid = pd.concat([OH_valid_cols, numerical_valid_cols], axis=1);
OH_test = pd.concat([OH_test_cols, numerical_test_cols], axis=1);

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder();
y_train = label_encoder.fit_transform(y_train);
print(y_val);
y_val = label_encoder.transform(y_val);
print(y_val);

from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(OH_train, y_train)
print(tpot.score(OH_valid, y_val))
tpot.export('tpot_attrition_pipeline.py')

from sklearn.linear_model import LogisticRegression
#Use LR to fit the data
lr = LogisticRegression(C=1e5);
lr.fit(OH_train, y_train);
my_valid_predict = lr.predict(OH_valid);

#error
false_number = ((my_valid_predict - y_val) ** 2).sum();
total_number = len(y_val);
print((total_number - false_number) / total_number);

from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
#Use LR to fit the data
lr = RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.3, min_samples_leaf=14, min_samples_split=5, n_estimators=100);
lr.fit(OH_train, y_train);
my_valid_predict = lr.predict(OH_valid);

#error
false_number = ((my_valid_predict - y_val) ** 2).sum();
total_number = len(y_val);
print((total_number - false_number) / total_number);

import numpy as np
test_preds = lr.predict_proba(OH_test)[:,1];
print(test_preds);
print(test_employee_data.index);
output = pd.DataFrame({'user_id': test_employee_data.index, 'Attrition': test_preds});
output.to_csv('submission5.csv', index=False);
print(output)

         Age Attrition  ... YearsSinceLastPromotion  YearsWithCurrManager
user_id                 ...                                              
1374      58        No  ...                       0                     0
1092      45        No  ...                       0                     3
768       40        No  ...                       7                     5
569       36        No  ...                       0                     9
911       25       Yes  ...                       1                     0

[5 rows x 35 columns]
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64
user_id
172      No
129      No
1033    Yes
178      No
535      No
       ... 
468      No
1342     No
352      No
865      No
275      No
Name: Attrition, Length: 236, dtype: object
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 0 

Generation 1 - Current best internal CV score: 0.8627659574468085
Generation 2 - Current best internal CV score: 0.8627659574468085
Generation 3 - Current best internal CV score: 0.8627659574468085
Generation 4 - Current best internal CV score: 0.8712765957446809
Generation 5 - Current best internal CV score: 0.8712765957446809

Best pipeline: GradientBoostingClassifier(RobustScaler(LinearSVC(input_matrix, C=5.0, dual=True, loss=hinge, penalty=l2, tol=0.01)), learning_rate=0.1, max_depth=3, max_features=0.1, min_samples_leaf=13, min_samples_split=18, n_estimators=100, subsample=0.1)
0.8728813559322034
0.864406779661017


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8601694915254238
[0.0950458  0.0622992  0.17648041 0.10993317 0.75166051 0.27708544
 0.21068689 0.13808487 0.05480523 0.28992204 0.09286446 0.13901118
 0.08421538 0.60189088 0.07400247 0.0249659  0.09715929 0.08656117
 0.0907369  0.14583807 0.49281294 0.08911559 0.04778366 0.0857273
 0.33746428 0.18616286 0.0500704  0.03280864 0.67306968 0.07413017
 0.06503824 0.07330848 0.17001727 0.1017624  0.06901357 0.06103356
 0.10382727 0.13206228 0.08691498 0.17564788 0.06276221 0.0671116
 0.08719808 0.10360249 0.08074929 0.54856314 0.21597966 0.01971509
 0.72874792 0.56509797 0.15617582 0.49842542 0.11382895 0.06671736
 0.3889534  0.08935377 0.05534907 0.08010666 0.04178079 0.33857237
 0.10233647 0.12035061 0.07360842 0.04188693 0.26632773 0.13374261
 0.34689123 0.06309981 0.0816085  0.14976784 0.08409813 0.49324887
 0.0635755  0.03923278 0.0706879  0.0618865  0.09765983 0.11124615
 0.39773027 0.03914431 0.0859575  0.03427851 0.07773309 0.04249799
 0.10452656 0.04374978 0.06296349 0.14485392 